In [1]:
# --- CELL 1: Imports and setup ---
import pandas as pd
import numpy as np
import cv2
import os

# Define your base paths
csv_input = "detected_plates_all.csv"
video_path = "/Users/ilakkiat/Documents/MIT/SEM-V/CV-LAB/CV-PROJECT/CODE/data/cropped_10.mp4"
interpolated_csv = "/Users/ilakkiat/Documents/MIT/SEM-V/CV-LAB/CV-PROJECT/CODE/outputs/interpolated_results.csv"
output_video = "/Users/ilakkiat/Documents/MIT/SEM-V/CV-LAB/CV-PROJECT/CODE/outputs/final_overlay.mp4"

os.makedirs("outputs", exist_ok=True)
print("Paths ready ✅")


Paths ready ✅


In [3]:
# --- CELL 2: Interpolation function ---
def interpolate_boxes(csv_path, output_path):
    df = pd.read_csv(csv_path)
    df = df.sort_values(by=["Plate", "Frame"]).reset_index(drop=True)
    interpolated_rows = []

    for plate, group in df.groupby("Plate"):
        group = group.sort_values("Frame").reset_index(drop=True)
        for i in range(len(group) - 1):
            row1, row2 = group.iloc[i], group.iloc[i + 1]
            f1, f2 = int(row1["Frame"]), int(row2["Frame"])
            frame_gap = f2 - f1

            interpolated_rows.append(row1)

            if frame_gap > 1:
                for f in range(1, frame_gap):
                    alpha = f / frame_gap
                    interp = {
                        "Frame": f1 + f,
                        "Car_X": row1["Car_X"] + alpha * (row2["Car_X"] - row1["Car_X"]),
                        "Car_Y": row1["Car_Y"] + alpha * (row2["Car_Y"] - row1["Car_Y"]),
                        "Car_W": row1["Car_W"] + alpha * (row2["Car_W"] - row1["Car_W"]),
                        "Car_H": row1["Car_H"] + alpha * (row2["Car_H"] - row1["Car_H"]),
                        "Plate_X": row1["Plate_X"] + alpha * (row2["Plate_X"] - row1["Plate_X"]),
                        "Plate_Y": row1["Plate_Y"] + alpha * (row2["Plate_Y"] - row1["Plate_Y"]),
                        "Plate_W": row1["Plate_W"] + alpha * (row2["Plate_W"] - row1["Plate_W"]),
                        "Plate_H": row1["Plate_H"] + alpha * (row2["Plate_H"] - row1["Plate_H"]),
                        "Timestamp": np.nan,
                        "Plate": plate
                    }
                    interpolated_rows.append(interp)

        interpolated_rows.append(group.iloc[-1].to_dict())


    df_interp = pd.DataFrame(interpolated_rows)
    df_interp.to_csv(output_path, index=False)
    print(f"✅ Interpolated CSV saved to {output_path}")
    return df_interp

# Run interpolation
df_interp = interpolate_boxes(csv_input, interpolated_csv)
df_interp.head()


✅ Interpolated CSV saved to /Users/ilakkiat/Documents/MIT/SEM-V/CV-LAB/CV-PROJECT/CODE/outputs/interpolated_results.csv


,Frame,Car_X,Car_Y,Car_W,Car_H,Plate_X,Plate_Y,Plate_W,Plate_H,Timestamp,Plate
0,183,180.00,46.0,500.00,412.00,1.0,1.00,476.0,69.00,12:40:18,IG1660
1,199,127.00,145.0,575.00,443.00,25.0,2.00,10.0,15.00,12:40:20,IG6660
2,180,185.00,31.0,490.00,400.00,446.0,2.00,44.0,25.00,12:40:18,LG1660
3,181,183.25,36.5,493.25,403.25,335.0,1.75,154.5,34.25,NaN,LG1660
4,182,181.50,42.0,496.50,406.50,224.0,1.50,265.0,43.50,NaN,LG1660


In [4]:
# --- CELL 3: Draw annotations on video ---
def draw_annotations(video_path, csv_path, output_path):
    df = pd.read_csv(csv_path)
    cap = cv2.VideoCapture(video_path)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    fps = cap.get(cv2.CAP_PROP_FPS)
    w, h = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    out = cv2.VideoWriter(output_path, fourcc, fps, (w, h))

    frame_dict = {int(row["Frame"]): row for _, row in df.iterrows()}
    frame_idx = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame_idx += 1

        if frame_idx in frame_dict:
            r = frame_dict[frame_idx]
            x, y, w_, h_ = map(int, [r["Car_X"], r["Car_Y"], r["Car_W"], r["Car_H"]])
            xp, yp, wp, hp = map(int, [r["Plate_X"], r["Plate_Y"], r["Plate_W"], r["Plate_H"]])
            plate_text = str(r["Plate"])

            cv2.rectangle(frame, (x, y), (x + w_, y + h_), (0, 255, 0), 2)
            cv2.rectangle(frame, (x + xp, y + yp), (x + xp + wp, y + yp + hp), (0, 0, 255), 2)
            cv2.putText(frame, plate_text, (x, y - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

        out.write(frame)

    cap.release()
    out.release()
    print(f"✅ Annotated video saved to {output_path}")

# Run overlay
draw_annotations(video_path, interpolated_csv, output_video)


✅ Annotated video saved to /Users/ilakkiat/Documents/MIT/SEM-V/CV-LAB/CV-PROJECT/CODE/outputs/final_overlay.mp4


In [5]:
# --- CELL 4: Summary ---
import IPython.display as display
print("✅ Postprocessing complete.")
display.display(df_interp.head())


✅ Postprocessing complete.


,Frame,Car_X,Car_Y,Car_W,Car_H,Plate_X,Plate_Y,Plate_W,Plate_H,Timestamp,Plate
0,183,180.00,46.0,500.00,412.00,1.0,1.00,476.0,69.00,12:40:18,IG1660
1,199,127.00,145.0,575.00,443.00,25.0,2.00,10.0,15.00,12:40:20,IG6660
2,180,185.00,31.0,490.00,400.00,446.0,2.00,44.0,25.00,12:40:18,LG1660
3,181,183.25,36.5,493.25,403.25,335.0,1.75,154.5,34.25,NaN,LG1660
4,182,181.50,42.0,496.50,406.50,224.0,1.50,265.0,43.50,NaN,LG1660


In [ ]:
import cv2

cap = cv2.VideoCapture(output_video)
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    cv2.imshow("Interpolated Output", frame)
    if cv2.waitKey(30) & 0xFF == 27:
        break
cap.release()
cv2.destroyAllWindows()


: 

Run Flow

Place this notebook inside your CODE/ folder.

Ensure you have detected_plates_all.csv already saved.

Run all cells sequentially.

You’ll get:

outputs/interpolated_results.csv

outputs/final_overlay.mp4 with all bounding boxes + plates.